# Running Mistral 7B on Google Colab with GPU

This notebook demonstrates how to run the Mistral 7B model using Google Colab's GPU capabilities for the domain suggestion task.

## 1. Setup Environment

First, let's install the required dependencies:

In [ ]:
!pip install -q torch transformers accelerate

## 2. Check GPU Availability

Let's check if we have access to a GPU:

In [ ]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")

## 3. Load the Mistral 7B Model

We'll use the Mistral-7B-Instruct-v0.2 model which is well-suited for instruction following tasks:

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Model ID for Mistral 7B Instruct v0.2
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load model with optimizations for Colab
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto"
)

## 4. Define Domain Suggestion Function

Let's create a function that uses the Mistral model to generate domain suggestions:

In [ ]:
def generate_domain_suggestions(business_description, num_suggestions=5):
    """
    Generate domain name suggestions for a business description using Mistral 7B.
    """
    # Create a prompt for the model
    prompt = f"""[INST] You are a domain name suggestion expert. Based on the business description, suggest {num_suggestions} domain names with different extensions (.com, .net, .io, .co, .ai, etc.).

Business Description: {business_description}

Provide your suggestions in the following format:
1. domain1.com (confidence: 0.95)
2. domain2.net (confidence: 0.87)
...
[/INST]"""

    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate response
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=500,
            temperature=0.7,
            top_p=0.9,
            do_sample=True
        )

    # Decode the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract just the assistant's response
    start_marker = "[/INST]"
    if start_marker in response:
        response = response.split(start_marker)[1].strip()
    
    return response

## 5. Test the Domain Suggestion System

Let's test our domain suggestion system with some example business descriptions:

In [ ]:
# Example business descriptions
test_businesses = [
    "A bakery that specializes in artisanal sourdough bread and pastries",
    "An online platform for learning coding and software development skills",
    "A sustainable fashion brand that creates eco-friendly clothing from recycled materials",
    "A mobile app for tracking fitness goals and connecting with personal trainers"
]

# Generate suggestions for each business
for i, business in enumerate(test_businesses, 1):
    print(f"\n{i}. Business: {business}\n")
    suggestions = generate_domain_suggestions(business)
    print(f"Domain Suggestions:\n{suggestions}\n")
    print("-" * 80)

## 6. Optimized Version for Better Performance

Let's create a more optimized version that uses batching for better performance:

In [ ]:
def generate_domain_suggestions_batch(business_descriptions, num_suggestions=5):
    """
    Generate domain name suggestions for multiple business descriptions using batch processing.
    """
    # Create prompts for all business descriptions
    prompts = []
    for business in business_descriptions:
        prompt = f"""[INST] You are a domain name suggestion expert. Based on the business description, suggest {num_suggestions} domain names with different extensions (.com, .net, .io, .co, .ai, etc.).

Business Description: {business}

Provide your suggestions in the following format:
1. domain1.com (confidence: 0.95)
2. domain2.net (confidence: 0.87)
...
[/INST]"""
        prompts.append(prompt)

    # Tokenize all prompts
    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to("cuda")

    # Generate responses
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=500,
            temperature=0.7,
            top_p=0.9,
            do_sample=True
        )

    # Decode responses
    responses = []
    for output in outputs:
        response = tokenizer.decode(output, skip_special_tokens=True)
        # Extract just the assistant's response
        start_marker = "[/INST]"
        if start_marker in response:
            response = response.split(start_marker)[1].strip()
        responses.append(response)
    
    return responses

## 7. Testing the Batch Version

Let's test our optimized batch version:

In [ ]:
# Test the batch version
print("Testing batch processing...")
batch_responses = generate_domain_suggestions_batch(test_businesses)

for i, (business, response) in enumerate(zip(test_businesses, batch_responses), 1):
    print(f"\n{i}. Business: {business}\n")
    print(f"Domain Suggestions:\n{response}\n")
    print("-" * 80)

## 8. Fine-tuning for Domain Suggestion Task

If you want to fine-tune the model specifically for domain suggestion, here's a basic approach:

In [ ]:
# Install additional libraries for fine-tuning
!pip install -q peft bitsandbytes

In [ ]:
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments, Trainer

# Configuration for LoRA fine-tuning
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA to the model
lora_model = get_peft_model(model, lora_config)
lora_model.print_trainable_parameters()

## 9. Conclusion

This notebook demonstrated how to:
1. Run Mistral 7B on Google Colab with GPU acceleration
2. Generate domain name suggestions using the model
3. Optimize performance with batch processing
4. Prepare for fine-tuning the model for the specific domain suggestion task

To use this in production, you would want to:
1. Fine-tune the model on your specific domain suggestion dataset
2. Implement additional post-processing to ensure domain name validity
3. Add a database of already registered domains to filter out unavailable names
4. Implement rate limiting and caching for better performance